# CENG0036 CW - Group I
## Section 1 - Introduction


This notebook is a supplement to the coursework report.

The purpose of the codes below is to showcase how to replicate our code in order to arrive at our presented predictions of outcomes of the matches that will take place on the weekend of 1st February 2025.

Note that Python 3.9.6 was used in building the codes.

### 1.1 - Importing Libraries
Below are all of the libraries used in developing the entirety of the codes that follow.

In [2]:
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from datetime import timedelta
from collections import deque, defaultdict

## Section 2 - Data Import

### 2.1 - Scrutinising the initial epl-training dataframe
The following code was used to identify any duplicates/empty rows in the initital dataset.

In [3]:
epl = pd.read_csv('epl-training.csv')

NSeasons = 24
MatchesPerSeason = 380
ExpectedLength = NSeasons*MatchesPerSeason

print(f'Length of epl before is {len(epl)}, which is {len(epl)-ExpectedLength} longer than expected')

#removing empty rows
epl = epl.dropna(how='all')

duplicates = epl.iloc[:, :3].duplicated()
if duplicates.any():
    dupindex = duplicates[duplicates].index.tolist()
    print(f"Duplicate rows found at indices {dupindex}")
    print(f'there are {len(dupindex)} duplicates')
else:
    print("All rows in the first three columns of epl are unique")

# Eliminate duplicate rows with the indices stored in dupindex
epl = epl.drop(dupindex)
print(f"Duplicate rows have been removed and length is now {len(epl)}")

Length of epl before is 9221, which is 101 longer than expected
Duplicate rows found at indices [8841, 8842, 8843, 8844, 8845, 8846, 8847, 8848, 8849, 8850, 8851, 8852, 8853, 8854, 8855, 8856, 8857, 8858, 8859, 8860, 8861, 8862, 8863, 8864, 8865, 8866, 8867, 8868, 8869, 8870, 8871, 8872, 8873, 8874, 8875, 8876, 8877, 8878, 8879, 8880, 8881, 8882, 8883, 8884, 8885, 8886, 8887, 8888, 8889, 8890, 8891, 8892, 8893, 8894, 8895, 8896, 8897, 8898, 8899, 8900, 8901, 8902, 8903, 8904, 8905, 8906, 8907, 8908, 8909, 8910, 8911, 8912, 8913, 8914, 8915, 8916, 8917, 8918, 8919, 8920, 8921, 8922, 8923, 8924, 8925, 8926, 8927, 8928, 8929, 8930, 8931, 8932, 8933, 8934, 8935, 8936, 8937, 8938, 8939, 8940]
there are 100 duplicates
Duplicate rows have been removed and length is now 9120


### 2.2 - Web Scraping

The following is an example code of the webscraping task to retrieve EPL team market values across the years. Note that the same function, with tweaks in some of the parameters, including URL, table class, and row_data keys, was used to webscrape all the other external features, inlcuding match dates across different competitions, posession, attendance, set piece, etc.

```py
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

#(year_i=2017) == (year_i =2017-2018 season)
def get_market_val(year_i, year_f):
    #Fake user agent to avoid 403 forbidden error
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    combined_df = pd.DataFrame()
    
    for year in range(year_i,year_f):
        data_list = []
        print(year)
        url = f"https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id={year}#google_vignette"
        response = requests.get(url, headers=headers)
        print(f'status code is: {response.status_code}')
        soup = BeautifulSoup(response.text, 'html.parser')

        table = soup.find('table', class_ = 'items')
        rows = table.find('tbody').find_all('tr')

        #Loop through the rows of the table
        for row in rows:
            columns = row.find_all('td')

            row_data = {
                'Year': year,
                'Club': columns[1].text.strip(),
                'TMV': columns[6].text.strip()[1:],  
            }

            data_list.append(row_data)

        df = pd.DataFrame(data_list)

        #Remove empty rows
        df = df[~df.apply(lambda row:row.astype(str).str.strip().eq('').all(), axis=1)]
        combined_df = pd.concat([combined_df, df], ignore_index=True)
        time.sleep(5)


    return combined_df.to_csv('Engineered Data/Final Data/marketval.csv', index=False)

get_market_val(2000, 2025)
```

### 2.3 - Merging Scraped Data

2.3.1 - Adding the 14-Day Match Density & Attendance data:

In [4]:
#Importing data scraping files (EPL teams' matches in other comptetitions)
EPL_S = pd.read_csv('Scraped Data/COMBINED_EPL.csv') #this is a similar dataframe to epltraining, yet it has the advantage of having attendance data
FA_S = pd.read_csv('Scraped Data/COMBINED_FA_E.csv')
EFL_S = pd.read_csv('Scraped Data/COMBINED_EFL_E.csv')
UCL_S = pd.read_csv('Scraped Data/COMBINED_UCL_E.csv')
UEL_S = pd.read_csv('Scraped Data/COMBINED_UEL_E.csv')
#clean epl-training (epl) file is the result of the previous code cell.
cleanepl = epl

#Combining data scraping files into a single dataframe
EPL_S.insert(0, 'df name', 'EPL')
FA_S.insert(0, 'df name', 'FA')
EFL_S.insert(0, 'df name', 'EFL')
UCL_S.insert(0, 'df name', 'UCL')
UEL_S.insert(0, 'df name', 'UEL')
combined = pd.concat([EPL_S, FA_S, EFL_S, UCL_S, UEL_S])

#Just to ensure the correct date format
cleanepl['Date'] = pd.to_datetime(cleanepl['Date'])
combined['Date'] = pd.to_datetime(combined['Date'])

combined.sort_values(['Date','HomeTeam'], ascending=[True, True], inplace=True)

#Calculating A14/H14 "14-day match density" algorithm:
def calculate_matches(team, match_date):
    StartDate = match_date - timedelta(days=14)
    matches = combined[
        ((combined["HomeTeam"]==team) | (combined['AwayTeam']==team)) &
        (combined['Date'] >= StartDate) &
        (combined['Date'] < match_date)
        ]
    return len(matches)

combined_epl = combined[combined['df name'] == 'EPL']

combined_epl['H14'] = combined_epl.apply(lambda row: calculate_matches(row['HomeTeam'], row['Date']), axis=1)
combined_epl['A14'] = combined_epl.apply(lambda row: calculate_matches(row['AwayTeam'], row['Date']), axis=1)

#Merging the combined dataframe with the clean epltraining dataframe to add the A14/H14 + Attendance columns
cleanepl = cleanepl.merge(
    combined_epl[['Date', 'HomeTeam', 'AwayTeam', 'H14', 'A14', 'Attendance']],
    on= ['Date', 'HomeTeam', 'AwayTeam'],
    how='left'
)

#Fixing the Date format
cleanepl['Date'] = cleanepl['Date'].dt.strftime('%d/%m/%Y')
print(cleanepl.head())

/var/folders/ng/c4n7kkzs4csgc8h21qrz97580000gn/T/ipykernel_22741/1866765347.py:19: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cleanepl['Date'] = pd.to_datetime(cleanepl['Date'])
/var/folders/ng/c4n7kkzs4csgc8h21qrz97580000gn/T/ipykernel_22741/1866765347.py:20: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  combined['Date'] = pd.to_datetime(combined['Date'])
/var/folders/ng/c4n7kkzs4csgc8h21qrz97580000gn/T/ipykernel_22741/1866765347.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_epl['H14'] = combined_

         Date  HomeTeam       AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0  19/08/2000  Charlton       Man City   4.0   0.0   H   2.0   0.0   H   
1  19/08/2000   Chelsea       West Ham   4.0   2.0   H   1.0   0.0   H   
2  19/08/2000  Coventry  Middlesbrough   1.0   3.0   A   1.0   1.0   D   
3  19/08/2000     Derby    Southampton   2.0   2.0   D   1.0   2.0   A   
4  19/08/2000     Leeds        Everton   2.0   0.0   H   2.0   0.0   H   

            Referee  ...   AC    HF    AF   HY   AY   HR   AR  H14  A14  \
0        Rob Harris  ...  6.0  13.0  12.0  1.0  2.0  0.0  0.0    0    0   
1     Graham Barber  ...  7.0  19.0  14.0  1.0  2.0  0.0  0.0    0    0   
2      Barry Knight  ...  4.0  15.0  21.0  5.0  3.0  1.0  0.0    0    0   
3       Andy D'Urso  ...  8.0  11.0  13.0  1.0  1.0  0.0  0.0    0    0   
4  Dermot Gallagher  ...  4.0  21.0  20.0  1.0  3.0  0.0  0.0    0    0   

   Attendance  
0      20,043  
1      34,914  
2      20,624  
3      27,223  
4      40,010  

[5 rows

/var/folders/ng/c4n7kkzs4csgc8h21qrz97580000gn/T/ipykernel_22741/1866765347.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_epl['A14'] = combined_epl.apply(lambda row: calculate_matches(row['AwayTeam'], row['Date']), axis=1)


2.3.2 - Adding the Referee Strictness feature:

In [5]:
# continuing cleanepl from the previous cell under the variable (strepl)
strepl = cleanepl

#Standardising referee names:
def StandardNames(index, name):
    if index <= 379:
        parts = name.split()
        if len(parts) > 1:
            return f"{parts[0][0]} {parts[1]}" #double check
        else:
            return name
        
    elif index <= 549:
        parts = name.replace('.','').split()
        if len(parts) > 1:
            return f"{parts[0][0]} {parts[-1]}"
        else:
            return name
    
    elif index <= 759:
        parts = name.replace(',','').replace('.','').split()
        if len(parts) > 1:
            return f"{parts[1][0]} {parts[0]}"
        else:
            return name
    elif index >= 1855 and index <= 1863:
        parts = name.split()
        if len(parts) > 1:
            return f"{parts[0][-1]} {parts[1]}"
        else:
            return name
    else:
        return name
        
strepl['Referee'] = strepl.apply(lambda row: StandardNames(row.name, row['Referee']), axis=1) 

# Define a lookup dictionary for inconsistent names
name_corrections = {
    "D Gallaghe": "D Gallagher",
    "D Gallagh": "D Gallagher"
}

# Apply corrections to the 'Referee' column
strepl['Referee'] = strepl['Referee'].apply(lambda name: name_corrections[name] if name in name_corrections else name)


#Initialising dictionaries to 0
refs = strepl['Referee'].unique() 
Y = {ref: 0 for ref in refs} 
R = {ref: 0 for ref in refs}
MatchCount = {ref: 0 for ref in refs}
strictness = {}

#Loop to count referee stats
for index, row in strepl.iterrows(): 
    ref = row['Referee']
    if pd.notna(ref):
        Y[ref] += row['AY'] + row['HY']
        R[ref] += row['AY'] + row['HY']
        MatchCount[ref] += 1

#Loop to evaluate referee strictness
for ref in refs:
    if MatchCount[ref] >0:
        strictness[ref] = (Y[ref] + 3*R[ref])/MatchCount[ref]
    else:
        strictness[ref] = 0

#Add to dataframe and add to CSV
strepl['Strictness'] = strepl['Referee'].map(strictness)

print(strepl.head())

         Date  HomeTeam       AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0  19/08/2000  Charlton       Man City   4.0   0.0   H   2.0   0.0   H   
1  19/08/2000   Chelsea       West Ham   4.0   2.0   H   1.0   0.0   H   
2  19/08/2000  Coventry  Middlesbrough   1.0   3.0   A   1.0   1.0   D   
3  19/08/2000     Derby    Southampton   2.0   2.0   D   1.0   2.0   A   
4  19/08/2000     Leeds        Everton   2.0   0.0   H   2.0   0.0   H   

       Referee  ...    HF    AF   HY   AY   HR   AR  H14  A14  Attendance  \
0     R Harris  ...  13.0  12.0  1.0  2.0  0.0  0.0    0    0      20,043   
1     G Barber  ...  19.0  14.0  1.0  2.0  0.0  0.0    0    0      34,914   
2     B Knight  ...  15.0  21.0  5.0  3.0  1.0  0.0    0    0      20,624   
3     A D'Urso  ...  11.0  13.0  1.0  1.0  0.0  0.0    0    0      27,223   
4  D Gallagher  ...  21.0  20.0  1.0  3.0  0.0  0.0    0    0      40,010   

   Strictness  
0   15.272727  
1   13.641026  
2   12.253968  
3   12.565657  
4   10.11023

2.3.3 - Adding the Standings Feature:

In [6]:
tqdm.pandas()

# continuing strepl from the previous cell under the variable (stepl)
stepl = strepl

#Changing the FTR columns to indicate the name of the winner to simplify the codes below
stepl['FTR'] = stepl.apply(lambda row: row['HomeTeam'] if row['FTR']=='H' else ('Draw' if row['FTR'] == 'D' else row['AwayTeam']), axis=1)

#Adding season round # values
stepl['Season'] = stepl.index // 380
stepl['Season'] = stepl['Season'].apply(lambda i: 2000 + i)

roundindex = (stepl.index - 10) // 10 + 1
stepl['Round'] = (roundindex % 38) + 1 

#Calculation algorithm of team points across rounds and seasons
def get_pts(team, season, round):
    '''
    e.g. (season = 2018) == (season = 2018-2019)
    '''
    if round == 1:
        return 0
    
    prevround = stepl[
        (stepl['Season'] == season) & 
        (stepl['Round'] == round-1)
    ]
    
    #Checking if the team won:
    homewin = (prevround['HomeTeam'] == team) & (prevround['FTR'] == team)
    awaywin = (prevround['AwayTeam'] == team) & (prevround['FTR'] == team)
    draw = ((prevround['HomeTeam'] == team) | (prevround['AwayTeam'] == team)) & (prevround['FTR'] == 'Draw')

    if homewin.any() or awaywin.any():
        roundpts = 3
    elif draw.any():
        roundpts = 1
    else:
        roundpts = 0
    
    return roundpts + get_pts(team, season, round-1)


stepl['Hpts'] = stepl.progress_apply(lambda row: get_pts(row['HomeTeam'], row['Season'], row['Round']), axis=1)
stepl['Apts'] = stepl.progress_apply(lambda row: get_pts(row['AwayTeam'], row['Season'], row['Round']), axis=1)

#Reversing the FTR column into what it used to be
stepl['FTR'] = stepl.progress_apply(lambda row: 'H' if row['FTR']==row['HomeTeam'] else ('D' if row['FTR']=='Draw' else 'A'), axis=1)
print(stepl.head())

100%|██████████| 9120/9120 [00:00<00:00, 207607.26it/s]

         Date  HomeTeam       AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0  19/08/2000  Charlton       Man City   4.0   0.0   H   2.0   0.0   H   
1  19/08/2000   Chelsea       West Ham   4.0   2.0   H   1.0   0.0   H   
2  19/08/2000  Coventry  Middlesbrough   1.0   3.0   A   1.0   1.0   D   
3  19/08/2000     Derby    Southampton   2.0   2.0   D   1.0   2.0   A   
4  19/08/2000     Leeds        Everton   2.0   0.0   H   2.0   0.0   H   

       Referee  ...   HR   AR  H14  A14  Attendance  Strictness  Season  \
0     R Harris  ...  0.0  0.0    0    0      20,043   15.272727    2000   
1     G Barber  ...  0.0  0.0    0    0      34,914   13.641026    2000   
2     B Knight  ...  1.0  0.0    0    0      20,624   12.253968    2000   
3     A D'Urso  ...  0.0  0.0    0    0      27,223   12.565657    2000   
4  D Gallagher  ...  0.0  0.0    0    0      40,010   10.110236    2000   

   Round  Hpts  Apts  
0      1     0     0  
1      1     0     0  
2      1     0     0  
3      1    

2.3.4 Adding the following metrics: team strength, goals scored record, defensive strength, home form points, goal difference form, win streak, and head to head win rate for both the home and away teams. Note that the definition of each metric is discussed in more details in the report.

In [7]:
# continuing stepl from the previous cell under the variable (comepl)
comepl = stepl

# Temporarily add the HomePoints/AwayPoints features to simplify the codes that follow
# Points calculation: Win = 3, Draw = 1, Loss = 0
comepl['HomePoints'] = comepl['FTR'].apply(lambda x: 3 if x == 'H' else (1 if x == 'D' else 0))
comepl['AwayPoints'] = comepl['FTR'].apply(lambda x: 3 if x == 'A' else (1 if x == 'D' else 0))

# Define rolling window size for Form Metrics (e.g., last 10 matches)
form_window = 10

# Dictionaries to track team stats and rolling metrics
team_strength_stats = {}
team_goals_scored = {}
team_goals_conceded = {}
team_form_points = {}
team_form_goal_diff = {}
team_win_streak = {}

# H2H tracking dictionary
h2h_record = defaultdict(lambda: {"matches": 0, "home_wins": 0, "away_wins": 0})

# Helper functions for Team Strength (overall metrics)
def get_team_strength(team):
    if team in team_strength_stats and team_strength_stats[team]['games'] > 0:
        return team_strength_stats[team]['points'] / team_strength_stats[team]['games']
    else:
        return 0

def update_team_strength(team, points):
    if team not in team_strength_stats:
        team_strength_stats[team] = {'points': 0, 'games': 0}
    team_strength_stats[team]['points'] += points
    team_strength_stats[team]['games'] += 1

def get_goal_scoring_rate(team):
    if team in team_goals_scored and team_strength_stats[team]['games'] > 0:
        return team_goals_scored[team] / team_strength_stats[team]['games']
    else:
        return 0

def update_goal_scoring_rate(team, goals):
    if team not in team_goals_scored:
        team_goals_scored[team] = 0
    team_goals_scored[team] += goals

def get_defensive_strength(team):
    if team in team_goals_conceded and team_strength_stats[team]['games'] > 0:
        return team_goals_conceded[team] / team_strength_stats[team]['games']
    else:
        return 0

def update_defensive_strength(team, goals_conceded):
    if team not in team_goals_conceded:
        team_goals_conceded[team] = 0
    team_goals_conceded[team] += goals_conceded

# Helper functions for Team Form (last `form_window` matches)
def get_form_points(team):
    if team in team_form_points and len(team_form_points[team]) > 0:
        return sum(team_form_points[team]) / len(team_form_points[team])
    else:
        return 0

def update_form_points(team, points):
    if team not in team_form_points:
        team_form_points[team] = deque(maxlen=form_window)
    team_form_points[team].append(points)

def get_goal_diff_form(team):
    if team in team_form_goal_diff and len(team_form_goal_diff[team]) > 0:
        return sum(team_form_goal_diff[team])
    else:
        return 0

def update_goal_diff_form(team, goal_diff):
    if team not in team_form_goal_diff:
        team_form_goal_diff[team] = deque(maxlen=form_window)
    team_form_goal_diff[team].append(goal_diff)

def get_win_streak(team):
    if team in team_win_streak:
        return team_win_streak[team]
    else:
        return 0

def update_win_streak(team, result):
    if team not in team_win_streak:
        team_win_streak[team] = 0
    if result == 3:  # Win
        team_win_streak[team] += 1
    else:  # Loss or Draw
        team_win_streak[team] = 0

# H2H functions
def calculate_h2h_win_rate(team, opponent, is_home):
    record = h2h_record[(team, opponent)]
    wins = record['home_wins'] if is_home else record['away_wins']
    total_matches = record['matches']
    return wins / total_matches if total_matches > 0 else 0

def update_h2h_record(home_team, away_team, result):
    h2h_record[(home_team, away_team)]['matches'] += 1
    h2h_record[(away_team, home_team)]['matches'] += 1
    if result == 'H':  # Home win
        h2h_record[(home_team, away_team)]['home_wins'] += 1
    elif result == 'A':  # Away win
        h2h_record[(away_team, home_team)]['away_wins'] += 1

# Add columns for Team Strength, Form, and H2H Metrics
hts_list = []
ats_list = []
home_gsr_list = []
away_gsr_list = []
home_ds_list = []
away_ds_list = []
home_form_points_list = []
away_form_points_list = []
home_goal_diff_form_list = []
away_goal_diff_form_list = []
home_win_streak_list = []
away_win_streak_list = []
home_h2h_win_rate_list = []
away_h2h_win_rate_list = []

for _, row in comepl.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']
    result = row['FTR']

    # Team Strength Metrics (over entire history)
    hts = get_team_strength(home_team)
    ats = get_team_strength(away_team)
    home_gsr = get_goal_scoring_rate(home_team)
    away_gsr = get_goal_scoring_rate(away_team)
    home_ds = get_defensive_strength(home_team)
    away_ds = get_defensive_strength(away_team)

    # Team Form Metrics (last `form_window` matches)
    home_form_points = get_form_points(home_team)
    away_form_points = get_form_points(away_team)
    home_goal_diff_form = get_goal_diff_form(home_team)
    away_goal_diff_form = get_goal_diff_form(away_team)
    home_win_streak = get_win_streak(home_team)
    away_win_streak = get_win_streak(away_team)

    # H2H Metrics
    home_h2h_win_rate = calculate_h2h_win_rate(home_team, away_team, is_home=True)
    away_h2h_win_rate = calculate_h2h_win_rate(away_team, home_team, is_home=False)

    # Append calculated values to lists
    hts_list.append(hts)
    ats_list.append(ats)
    home_gsr_list.append(home_gsr)
    away_gsr_list.append(away_gsr)
    home_ds_list.append(home_ds)
    away_ds_list.append(away_ds)
    home_form_points_list.append(home_form_points)
    away_form_points_list.append(away_form_points)
    home_goal_diff_form_list.append(home_goal_diff_form)
    away_goal_diff_form_list.append(away_goal_diff_form)
    home_win_streak_list.append(home_win_streak)
    away_win_streak_list.append(away_win_streak)
    home_h2h_win_rate_list.append(home_h2h_win_rate)
    away_h2h_win_rate_list.append(away_h2h_win_rate)

    # Update Team Strength stats
    update_team_strength(home_team, row['HomePoints'])
    update_team_strength(away_team, row['AwayPoints'])
    update_goal_scoring_rate(home_team, row['FTHG'])  # Goals scored
    update_goal_scoring_rate(away_team, row['FTAG'])  # Goals scored
    update_defensive_strength(home_team, row['FTAG'])  # Goals conceded
    update_defensive_strength(away_team, row['FTHG'])  # Goals conceded

    # Update Team Form stats
    home_goal_diff = row['FTHG'] - row['FTAG']  # Goal difference
    away_goal_diff = row['FTAG'] - row['FTHG']
    update_form_points(home_team, row['HomePoints'])
    update_form_points(away_team, row['AwayPoints'])
    update_goal_diff_form(home_team, home_goal_diff)
    update_goal_diff_form(away_team, away_goal_diff)
    update_win_streak(home_team, row['HomePoints'])
    update_win_streak(away_team, row['AwayPoints'])

    # Update H2H stats
    update_h2h_record(home_team, away_team, result)

# Assign calculated metrics to the dataframe
comepl['HTS'] = hts_list
comepl['ATS'] = ats_list
comepl['HGSR'] = home_gsr_list
comepl['AGSR'] = away_gsr_list
comepl['Home_DS'] = home_ds_list
comepl['Away_DS'] =away_ds_list
comepl['Home_Form_Points'] = home_form_points_list
comepl['Away_Form_Points'] = away_form_points_list
comepl['Home_Goal_Diff_Form'] = home_goal_diff_form_list
comepl['Away_Goal_Diff_Form'] = away_goal_diff_form_list
comepl['Home_Win_Streak'] = home_win_streak_list
comepl['Away_Win_Streak'] = away_win_streak_list
comepl['Home_H2H_Win_Rate'] = home_h2h_win_rate_list
comepl['Away_H2H_Win_Rate'] = away_h2h_win_rate_list

comepl.drop(columns=['HomePoints', 'AwayPoints'], inplace=True)

print(comepl.head())

         Date  HomeTeam       AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0  19/08/2000  Charlton       Man City   4.0   0.0   H   2.0   0.0   H   
1  19/08/2000   Chelsea       West Ham   4.0   2.0   H   1.0   0.0   H   
2  19/08/2000  Coventry  Middlesbrough   1.0   3.0   A   1.0   1.0   D   
3  19/08/2000     Derby    Southampton   2.0   2.0   D   1.0   2.0   A   
4  19/08/2000     Leeds        Everton   2.0   0.0   H   2.0   0.0   H   

       Referee  ...  Home_DS  Away_DS  Home_Form_Points  Away_Form_Points  \
0     R Harris  ...      0.0      0.0               0.0               0.0   
1     G Barber  ...      0.0      0.0               0.0               0.0   
2     B Knight  ...      0.0      0.0               0.0               0.0   
3     A D'Urso  ...      0.0      0.0               0.0               0.0   
4  D Gallagher  ...      0.0      0.0               0.0               0.0   

   Home_Goal_Diff_Form  Away_Goal_Diff_Form  Home_Win_Streak  Away_Win_Streak  \
0          

2.3.5: Finally, adding market value, possession stats, and set piece data for both the home and away teams:

In [8]:
MarketVal = pd.read_csv('Scraped Data/MarketValues.csv')
Posession = pd.read_csv('Scraped Data/PosessionData.csv')
SetPiece = pd.read_csv('Scraped Data/SetPiece.csv')

# continuing comepl from the previous cell under the variable (finalepl)
finalepl = comepl

Alterations = {
    'Manchester City': 'Man City',
    'Arsenal FC': 'Arsenal',
    'Chelsea FC': 'Chelsea',
    'Liverpool FC': 'Liverpool',
    'Manchester United': 'Man United',
    'Tottenham Hotspur': 'Tottenham',
    'Newcastle United': 'Newcastle',
    'Brighton & Hove Albion': 'Brighton',
    'West Ham United': 'West Ham',
    'Nottingham Forest': "Nott'm Forest",
    'Brentford FC': 'Brentford',
    'Wolverhampton Wanderers': 'Wolves',
    'AFC Bournemouth': 'Bournemouth',
    'Everton FC': 'Everton',
    'Fulham FC': 'Fulham',
    'Southampton FC': 'Southampton',
    'Leicester City': 'Leicester',
    'Ipswich Town': 'Ipswich',
    'West Bromwich Albion': 'West Brom',
    'Queens Park Rangers': 'QPR',
    'Hull City': 'Hull',
    'Stoke City': 'Stoke',
    'Swansea City': 'Swansea',
    'Manchester Utd': 'Man United',
    'Newcastle Utd': 'Newcastle',
    "Nott'ham Forest": "Nott'm Forest",
    "Luton Town": "Luton",
    'Sheffield Utd': 'Sheffield United',
    'Leeds United': 'Leeds',
    'Norwich City': 'Norwich',
    'Cardiff City': 'Cardiff',
    'Birmingham City': 'Birmingham',
    'Blackburn Rovers': 'Blackburn',
    'Blackpool FC': 'Blackpool',
    'Bolton Wanderers': 'Bolton',
    'Bradford City': 'Bradford',
    'Burnley FC': 'Burnley',
    'Charlton Athletic': 'Charlton',
    'Coventry City': 'Coventry',
    'Derby County': 'Derby',
    'Huddersfield Town': 'Huddersfield',
    'Middlesbrough FC': 'Middlesbrough',
    'Portsmouth FC': 'Portsmouth',
    'Reading FC': 'Reading',
    'Sunderland AFC': 'Sunderland',
    'Watford FC': 'Watford',
    'Wigan Athletic': 'Wigan',
               }

MarketVal['Club'] = MarketVal['Club'].apply(lambda name: Alterations[name] if name in Alterations else name)
SetPiece['Team'] = SetPiece['Team'].apply(lambda name: Alterations[name] if name in Alterations else name)
Posession['Team'] = Posession['Team'].apply(lambda name: Alterations[name] if name in Alterations else name)

# Checking if all names are covered in the dictionary
cleaneplteams = list(finalepl['HomeTeam'].unique())
MarketValteams = list(MarketVal['Club'].unique())
Posessionteams = list(Posession['Team'].unique())
setPieceteams = list(SetPiece['Team'].unique())
print(len(cleaneplteams),len(MarketValteams),len(Posessionteams), len(setPieceteams))
uniqueepl = sorted([team for team in cleaneplteams if team not in MarketValteams])
UniqueMarketVal = sorted([team for team in MarketValteams if team not in cleaneplteams])
UniquePosession = [team for team in Posessionteams if team not in cleaneplteams]
UniqueSetPiece = [team for team in setPieceteams if team not in cleaneplteams]

# Applying the dictionary to the dataframes
MarketVal['Club'] = MarketVal['Club'].apply(lambda name: Alterations[name] if name in Alterations else name)
SetPiece['Team'] = SetPiece['Team'].apply(lambda name: Alterations[name] if name in Alterations else name)
Posession['Team'] = Posession['Team'].apply(lambda name: Alterations[name] if name in Alterations else name)


# Cleaning the market value data
MarketVal['TMV'] = MarketVal['TMV'].apply(
    lambda value: float(str(value)[:-2]) * 1000  if isinstance(value, str) and value[-2:] == 'bn' else 
                  float(str(value)[:-1]) if isinstance(value, str) and value[-1] == 'm' else 
                  value
)

#Transferring market values to the epl df
finalepl = pd.merge(
    finalepl,
    MarketVal.rename(columns={'Club':'HomeTeam', 'TMV':'HTV($m)', 'Year':'Season'}),
    how='left',
    on=['HomeTeam', 'Season'],
)

finalepl = pd.merge(
    finalepl,
    MarketVal.rename(columns={'Club':'AwayTeam', 'TMV':'ATV($m)', 'Year':'Season'}),
    how='left',
    on=['AwayTeam', 'Season'],
)

# Transferring posession data to the clean epl df
Posession['year'] = Posession['year'].apply(lambda name: name[:4])
Posession['year'] = Posession['year'].astype(int)
Posession['Poss'] = Posession['Poss'].apply(lambda pos: pos/100)
Posession = Posession[['Team', 'Poss', 'year']]

finalepl = pd.merge(
    finalepl,
    Posession.rename(columns={'Team':'HomeTeam', 'Poss':'HTPos_avg', 'year':'Season'}),
    how='left',
    on= ['HomeTeam', 'Season']
)

finalepl = pd.merge(
    finalepl,
    Posession.rename(columns={'Team':'AwayTeam', 'Poss':'ATPos_avg', 'year':'Season'}),
    how='left',
    on= ['AwayTeam', 'Season']
)

#Transferring set piece values
SetPiece = SetPiece.dropna()
SetPiece['Season'] = SetPiece['Season'].apply(lambda year: year[:4]).astype(int)
SetPiece = SetPiece[['Season','Team','Set Piece Efficiency (%)','Penalty Efficiency (%)']]

finalepl = pd.merge(
    finalepl,
    SetPiece.rename(columns={'Team':'HomeTeam','Set Piece Efficiency (%)': 'HSPE (%)', 'Penalty Efficiency (%)': 'HPE (%)'}),
    how='left',
    on=['HomeTeam','Season']
)

finalepl = pd.merge(
    finalepl,
    SetPiece.rename(columns={'Team':'AwayTeam', 'Set Piece Efficiency (%)': 'ASPE (%)', 'Penalty Efficiency (%)': 'APE (%)'}),
    how='left',
    on=['AwayTeam','Season']
)

print(finalepl.head())

46 46 35 43
         Date  HomeTeam       AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0  19/08/2000  Charlton       Man City   4.0   0.0   H   2.0   0.0   H   
1  19/08/2000   Chelsea       West Ham   4.0   2.0   H   1.0   0.0   H   
2  19/08/2000  Coventry  Middlesbrough   1.0   3.0   A   1.0   1.0   D   
3  19/08/2000     Derby    Southampton   2.0   2.0   D   1.0   2.0   A   
4  19/08/2000     Leeds        Everton   2.0   0.0   H   2.0   0.0   H   

       Referee  ...  Home_H2H_Win_Rate  Away_H2H_Win_Rate  HTV($m)  ATV($m)  \
0     R Harris  ...                0.0                0.0      NaN      NaN   
1     G Barber  ...                0.0                0.0      NaN      NaN   
2     B Knight  ...                0.0                0.0      NaN      NaN   
3     A D'Urso  ...                0.0                0.0      NaN      NaN   
4  D Gallagher  ...                0.0                0.0      NaN      NaN   

   HTPos_avg  ATPos_avg  HSPE (%)  HPE (%)  ASPE (%)  APE (%)  
0   

In [9]:
finalepl.to_csv('alldata.csv')